In [ ]:
pip install pymysql

In [1]:
import pandas as pd  
pd.options.mode.chained_assignment = None 
import os, json, time, calendar, re
import pyodbc, pymysql    
import mysql.connector
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sqlalchemy import func, create_engine
from config import remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd 


mydb = mysql.connector.connect(  host=remote_db_endpoint,  user=remote_db_user,  password=remote_db_pwd,  database=remote_db_name) 
mycursor = mydb.cursor() 
sql="""
UPDATE ExpenseCategories 
  SET EC_PatternMapper='' 
  WHERE EC_PatternMapper IS NULL
"""
mycursor.execute(sql) 
mydb.commit() 

In [60]:
connstr=f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}"
pymysql.install_as_MySQLdb() 

In [61]:
pymysql.install_as_MySQLdb()
engine = create_engine(connstr) 
conn = engine.connect() 
dfcats = pd.read_sql("SELECT * FROM ExpenseCategories",con=conn)  
conn.close()
dfcats

,EC_ID,EC_Name,EC_Description,EC_PatternMapper,EC_AuditNotes
0,1,Travel,"commuting, general transportation",METRO|BIKE|UBER|LYFT|ZIPCAR|BICYCLE_LIGHT|BICY...,None
1,2,Vacation,vacation expenses,HOTEL|AIR|VACATION|RESORT|SHUTTLE|AIRBNB|VALEN...,None
2,3,Food & Drink,"bar and restaurant, alcohol",RESTAURANT|BAR|TONIC|LIQUOR|CENTRAL|WALGREENS|...,None
3,4,Shopping,general and retail purchases,DIGITAL_DEVICE_4_ACCESSORY|PORTABLE_ELECTRONIC...,None
4,5,Clothing,apparel,PANTS|DRESS|SHIRT|BOOT|SHOES|HAT|SOCKS|WATCH|P...,None
5,6,Groceries,food purchases,NUTRITION|TEA|SAUCE|SUGAR|DRINK|HERB|POPCORN|F...,None
6,7,Bills & Utilities,"billable services, basic utilities",YES ENERGY|MORTGAGE|CONDO FEE|TILTON BERNSTEIN...,None
7,8,Health & Wellness,"medical, care, excercise",MEDICAL|NUTRITIONAL_SUPPLEMENT|MEDICATION|SPOR...,None
8,9,Professional Services,contracted services,ETSY INC SELLER FEES|MARK GLOSSERMAN|DIGITAL T...,None
9,10,Entertainment,"entertainment subscriptions, movies, etc",SLING|AMAZON DIGIT|AUDIBLE|HBO|ABIS_BOOK|BOARD...,None


In [62]:
engine = create_engine(connstr) 
conn = engine.connect() 
dfcats = pd.read_sql("SELECT EC_ID, EC_Name, EC_PatternMapper FROM ExpenseCategories WHERE EC_PatternMapper IS NOT NULL AND EC_PatternMapper <> '' ",con=conn)  
conn.close()

In [63]:
def catReplace(c):
    c=str(c) 
    for i, row in dfcats.iterrows(): 
        if re.search(row.EC_PatternMapper.upper(),c.upper()):
            return row.EC_ID, row.EC_Name          
    return 19,'Misc'

In [49]:
print( catReplace('I AM safeway')[0]   )

6


In [64]:
def processDataDirectory(dir):
    files = os.listdir(f'Data/{dir}') 
    df = pd.DataFrame()
    for f in files:
        df1 = pd.read_csv(f'Data/{dir}/{f}')
        df = pd.concat([df1, df], axis=0, ignore_index=True)   
    df = df.rename(columns ={
                          'Order Date':'TransDate'
                         ,'Transaction Date':'TransDate'
                         ,'Item Total':'TransAmt'
                         ,'Amount':'TransAmt'
                         ,'Category':'TransCat'
                         ,'Title':'TransDesc' 
                         ,'Description':'TransDesc'})  
    df.TransDate=pd.to_datetime(df['TransDate']).dt.date
    df=df[['TransDate', 'TransDesc', 'TransCat', 'TransAmt']]
    df=df.drop_duplicates(subset=['TransDate','TransDesc','TransAmt'], keep="first")
    df['TransDesc'] = df['TransDesc'].fillna('')
    df['TransAmt'] = df['TransAmt'].apply(lambda x: str(x).replace('$',''))
    df['TransAmt'] = df['TransAmt'].astype('float').abs()  
    df['TransCat'] = df['TransDesc'].apply(lambda c: catReplace(c)[1] ) 
    df['TransCatId'] = df['TransDesc'].apply(lambda c: catReplace(c)[0] )     
    df['TransKey'] = df['TransDate'].map(lambda x: str(x)) 
    df['TransKey'] = df['TransKey'] +'-'+ df['TransAmt'].map(lambda x: str(x)) 
    df = df.reindex(columns=['TransDate', 'TransDesc', 'TransCat', 'TransCatId','TransAmt','TransKey'])  
    df.to_csv(f'Data/df_{dir}.csv', index=False) 
#     df.to_json(f'Data/df_{dir}.json', orient='records')
    return df


In [65]:
dirs = [f.path.replace('\\','/').replace('Data/','') for f in os.scandir('Data') if f.is_dir()]
df_tmas = pd.DataFrame();
for d in dirs:
    df = processDataDirectory(d)
    df_tmas = pd.concat([df_tmas, df], axis=0, ignore_index=True) 
df_tmas.to_csv(f'Data/df_tmas.csv', index=False)
df_tmas

,TransDate,TransDesc,TransCat,TransCatId,TransAmt,TransKey
0,2020-01-03,"Kernel Season's Popcorn Seasoning, Sour Cream ...",Groceries,6,11.88,2020-01-03-11.88
1,2020-01-09,wirarpa Women's High Waisted Cotton Underwear ...,Clothing,5,24.37,2020-01-09-24.37
2,2020-01-09,"Neutrogena Nourishing Eyeliner Pencil, Built-i...",Personal,13,7.92,2020-01-09-7.92
3,2020-01-09,Amazon Brand - Lark & Ro Women's Wide Leg Pont...,Clothing,5,37.10,2020-01-09-37.1
4,2020-01-09,Women's Elegant Vintage Cotton Casual Floral P...,Clothing,5,28.61,2020-01-09-28.61
...,...,...,...,...,...,...
1326,2020-01-02,WALMART.COM,Groceries,6,52.50,2020-01-02-52.5
1327,2020-01-02,UBER TRIP,Travel,1,8.00,2020-01-02-8.0
1328,2020-01-01,UBER TRIP,Travel,1,13.09,2020-01-01-13.09
1329,2019-12-31,7-ELEVEN 33452,Groceries,6,6.20,2019-12-31-6.2


In [66]:
engine = create_engine(connstr) 
conn = engine.connect() 
df_tmas.to_sql('TempTransTable', con=conn , if_exists='replace')
conn.close()


def terser(conetnt): 
    words=msg.split(' ') 
    stop_words = set(stopwords.words('english')) 
    filtered = []
    stemmer = PorterStemmer() 
    for word in words: 
        if word not in stop_words and word.isalpha(): 
            stemmed_word = stemmer.stem(word) 
            filtered.append(stemmed_word)
    return '-'.join(filtered)
msg = '''All work and not play makes jack a dull boy. To be or not to be.
nobody expects the spanish inquisition'''
print(terser(msg))
